In [1]:
import requests
import json
import forecastio
from pprint import pprint
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from Austin_airbnb import Austin_airbnb
import datetime as dt

importing Jupyter notebook from Austin_airbnb.ipynb


Austin_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
Austin_airbnb.ipynb:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "cells": [
Austin_airbnb.ipynb:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "metadata": {},


In [2]:
api_key = "a36c8345cc74c469a50078545e4970de"

In [4]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Austin, TX")

ATX_coord = []
ATX_coord.append(location.latitude)
ATX_coord.append(location.longitude)

ATX_coord

[30.2711286, -97.7436995]

In [6]:
forecast = forecastio.load_forecast(api_key, ATX_coord[0], ATX_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Light rain on Wednesday and Thursday, with high temperatures bottoming out at 58°F on Monday.
rain
71.66


In [7]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-21',
 '2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28']

In [8]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [10]:
ATX_table = pd.merge(df, Austin_airbnb, on="Date")
ATX_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-21,65.88,43.95,0.00,0.0000,none,75,370,234,120,63
1,2018-12-22,70.07,46.08,0.03,0.0005,rain,75,323,223,110,61
2,2018-12-23,59.38,41.15,0.07,0.0001,rain,97,408,196,93,58
3,2018-12-24,57.63,52.30,0.06,0.0003,rain,90,369,198,88,55
4,2018-12-25,71.66,64.42,0.23,0.0041,rain,92,361,197,88,57
5,2018-12-26,68.90,53.30,0.69,0.0184,rain,77,363,187,79,52
6,2018-12-27,64.60,47.75,0.73,0.0101,rain,77,347,153,58,47
7,2018-12-28,63.11,51.15,0.00,0.0000,none,57,264,184,76,41
